In [1]:
# importing libs
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.plotly as py
import plotly.graph_objs as go
import dash
import matplotlib.pyplot as plt
from helper_functions import *
from plotly.tools import FigureFactory as FF
data_dir = 'D:\Google_Drive\Machine_Learning\SpringBoard\Project I\data\\'

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [2]:
# importing data
# importing part d data (2014)
part_d_df = pd.read_csv(data_dir + 'cms_medicare_test\part_d_prescriber_2014000000000004.csv')
part_d_df = part_d_df[['nppes_provider_state', 'specialty_description', 'description_flag', 'drug_name', 'generic_name', 'bene_count', 'total_claim_count',\
                       'total_day_supply', 'total_drug_cost', 'bene_count_ge65', 'total_claim_count_ge65', 'total_day_supply_ge65', 'total_drug_cost_ge65']]
#part_d_df['nppes_provider_state'].isnull().sum()
part_d_df = part_d_df[part_d_df['nppes_provider_state'].isin(states)]

# importing census data (2014)
census_14_df = pd.read_csv(data_dir + 'US_Census65+\ACS_14_1YR_S0103_with_ann.csv', skiprows = [0, 1], \
                        names = ['Statename', 'totPop', '65Pop', 'Male', 'Female',  'MedAge', 'White', 'Black', 'Indian', 'Asian', 'Islander', 'OtherRace', 'MixedRace', 'Latino', 'WhiteOnly',\
                                '65DisPop', 'DisabilityPercent', '65PovPop', '100BelowPov', 'AtPov', 'AbovePoV'], \
                        usecols = [2, 3, 5, 9, 13, 17, 25, 29, 33, 37, 41, 46, 49, 53, 57, 173, 177, 337, 341, 345, 349])
census_14_df['Statename'] = census_14_df['Statename'].apply(statename_to_abbr)
census_14_df = census_14_df.sort_values(['Statename']).set_index(['Statename'])
#census_df.isnull().sum()

D:\Programs\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning:

Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.



# Drugs

In [3]:
# Dataprocessing
part_d_df['cost_per_bene'] = part_d_df['total_drug_cost']/part_d_df['bene_count']
part_d_df['claim_per_bene'] = part_d_df['total_claim_count']/part_d_df['bene_count']

part_d_df_state_mean = part_d_df.groupby(['nppes_provider_state']).mean().sort_values(['nppes_provider_state'])
part_d_df_state_sum = part_d_df.groupby(['nppes_provider_state']).sum().sort_values(['nppes_provider_state'])

part_d_df_state_sum['bene_per_65capt'] = part_d_df_state_sum['bene_count']/census_14_df['65Pop']

In [4]:
part_d_df_state_sum.head()

,bene_count,total_claim_count,total_day_supply,total_drug_cost,bene_count_ge65,total_claim_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,cost_per_bene,claim_per_bene,bene_per_65capt
nppes_provider_state,,,,,,,,,,,
AK,10111.0,104319,2889249,6.037885e+06,1247.0,43964.0,1321699.0,2.046262e+06,1.045900e+05,2232.647048,0.144652
AL,575461.0,2579117,96949467,1.770874e+08,180812.0,1420669.0,55849107.0,7.755902e+07,3.953995e+06,61749.009392,0.770866
AR,300545.0,1571436,54595898,9.865092e+07,84550.0,869589.0,31751614.0,4.150583e+07,2.041735e+06,38507.624819,0.646317
AZ,486609.0,1990948,87178551,1.572499e+08,116757.0,1016543.0,48113457.0,7.232470e+07,3.615874e+06,51449.814870,0.454453
CA,2565132.0,11556138,488137597,9.806524e+08,664881.0,6377563.0,274643276.0,4.953128e+08,2.158547e+07,289440.818924,0.514045


In [15]:
trace1 = go.Box(
    y=part_d_df_state_sum.bene_count_ge65/part_d_df_state_sum.bene_count*100,
    name = 'By No. of Beneficiaries',
    boxpoints='all',
    jitter=0.3,
    marker = dict(
        color = 'rgb(102,204,255)'
    ),
    boxmean=True
)

trace2 = go.Box(
    y=part_d_df_state_sum.total_claim_count_ge65/part_d_df_state_sum.total_claim_count*100,
    name = 'By No. of Claims',
    boxpoints='all',
    jitter=0.3,
    boxmean=True,
    marker = dict(
        color = 'rgb(051,153,255)'
    )
)

trace3 = go.Box(
    y=part_d_df_state_sum.total_drug_cost_ge65/part_d_df_state_sum.total_drug_cost*100,
    name = 'By Cost of Claims',
    boxpoints='all',
    jitter=0.3,
    boxmean=True,
    marker = dict(
        color='rgb(000,051,153)'
    )
)


layout = go.Layout(
    title='2014 Part D Medicare Used by Beneficiaries of 65+ Age',
    width=500,
    xaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
    ),
    yaxis=dict(
        title='Percentages (%)',
        zeroline=False
    ),
    legend=dict(
        orientation='h'
    ),
)

data = [trace1, trace2, trace3]
fig= go.Figure(data=data, layout=layout)
py.iplot(fig, filename='alcohol-box-plot')

In [34]:
part_d_df_state_sum.head()

,bene_count,total_claim_count,total_day_supply,total_drug_cost,bene_count_ge65,total_claim_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,cost_per_bene,claim_per_bene,bene_per_capt
nppes_provider_state,,,,,,,,,,,
AK,10111.0,104319,2889249,6.037885e+06,1247.0,43964.0,1321699.0,2.046262e+06,1.045900e+05,2232.647048,0.000000
AL,575461.0,2579117,96949467,1.770874e+08,180812.0,1420669.0,55849107.0,7.755902e+07,3.953995e+06,61749.009392,0.000000
AR,300545.0,1571436,54595898,9.865092e+07,84550.0,869589.0,31751614.0,4.150583e+07,2.041735e+06,38507.624819,0.000070
AZ,486609.0,1990948,87178551,1.572499e+08,116757.0,1016543.0,48113457.0,7.232470e+07,3.615874e+06,51449.814870,0.000000
CA,2565132.0,11556138,488137597,9.806524e+08,664881.0,6377563.0,274643276.0,4.953128e+08,2.158547e+07,289440.818924,0.001558


In [20]:
df_mean = part_d_df_state_mean.copy()
df_sum = part_d_df_state_sum.copy()

for col in df_mean.columns:
    df_mean[col] = df_mean[col].astype(int).astype(str)
    df_sum[col] = df_sum[col].astype(int).astype(str)

scl = [[0.0, 'rgb(204,255,255)'],[0.2, 'rgb(153,255,255)'],[0.4, 'rgb(102,204,255)'],\
            [0.6, 'rgb(051,153,255)'],[0.8, 'rgb(000,153,255)'],[1.0, 'rgb(000,051,153)']]

df_mean['text'] = df_mean.index + '<br>' +\
    'Avg. per Claim: '+ (df_sum['total_drug_cost'].astype(float).round(1)/df_sum['bene_count'].astype(float).round(1)).astype(str) +'<br>'\
    + 'Total No. Claims: '+ df_sum['total_claim_count']
    

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_mean.index,
        z = df_sum['total_drug_cost'].astype(float).round(1)/df_sum['total_claim_count'].astype(float).round(1),
        locationmode = 'USA-states',
        text = df_mean['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "USD")
        ) ]

layout = dict(
        title = '2014 Medicare (Part D) Average Cost per Beneficiary',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='part_d-cloropleth-map' )

In [56]:
def load_data_from_frac(file_name, directory, frac=True):
    
    """
    Function to load .csv files from Google BigQuery platform, which can come in partial files (frac = True) starting from 000000000000 after the file_name (no .csv needed)
    """
    
    list_ = []
    df = pd.DataFrame()
    
    if frac == True:
        i = 0
        while True:
            frac_no = str(i).zfill(12)
            my_file = Path(directory + file_name + frac_no + '.csv')
            if my_file.is_file():
                frame = pd.read_csv(my_file, low_memory=False)
                list_.append(frame)
                df = pd.concat(list_)
            else:
                break
            i += 1
    else:
        my_file = Path(directory + file_name + '000000000000' + '.csv')
        if my_file.is_file():
            frame = pd.read_csv(my_file, low_memory=False)
            list_.append(frame)
            df = pd.concat(list_)
            
    return df